# Chapter 13: Apriori Algorithm

โค้ดตัวอย่างการสร้างโมเดลประเภทการหากฎความสัมพันธ์ (Association Rules Learning) อย่างเช่น อัลกอริทึม Apriori  

ตัวอย่างนี้ใช้ชุดข้อมูล transactions การซื้อขายของขวัญสำหรับทุกโอกาส [`Groceries data`](http://archive.ics.uci.edu/ml/datasets/Online+Retail) โดยตัวอย่างนี้จะใช้ข้อมูลเฉพาะ transactions ทางออนไลน์ที่เกี่ยวกับประเทศ Sweden เท่านั้น และโค้ดถูกเขียนโดยใช้ไลบรารี่ [`mlxtend`](https://rasbt.github.io/mlxtend/)

**Reference**: https://www.geeksforgeeks.org/implementing-apriori-algorithm-in-python/

### ขั้นตอน
1. Import Package and Module
2. Load Dataset
3. Learn Association Rules
4. Rule Evaluation

## 1. Import Package and Module

ทำการนำเข้าแพ็กเกจและโมดูลที่จำเป็นต้องใช้อย่าง `mlxtend` ส่วน Pandas ใช้สำหรับบริหารจัดการข้อมูลตาราง

In [1]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

## 2. Load Dataset

ทำการโหลดชุดข้อมูล Groceries data เฉพาะ transactions การซื้อขายที่เกี่ยวกับประเทศ Sweden สำหรับตัวอย่างนี้ ซึ่งข้อมูลอยู่ใน Repository นี้

ข้อมูล 1 row แทน การซื้อสินค้า 1 ชนิดใน 1 transaction  แต่ละ transaction จะมี `Invoice No.` เฉพาะตัว  ส่วน `Description` เป็นชื่อสินค้า และ `Quantity` คือ จำนวนสินค้าที่ขายได้ในครั้งนั้น ซึ่งเราสนใจแค่ 3 ฟีเจอร์นี้เท่านั้น โดยเราจะมอง 1 สินค้า เป็น 1 item   ดังนั้น itemset ก็จะเป็นกลุ่มของสินค้าที่ถูกซื้อใน 1 transaction ที่สังเกตได้จาก `Invoice No.` ซึ่งเป็น transaction ID

In [2]:
data = pd.read_excel("https://github.com/sgulyano/DSI311/blob/main/Online%20Retail%20Sweden.xlsx?raw=true")
data.drop(['StockCode', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country'], axis=1, inplace=True)
data

,InvoiceNo,Description,Quantity
0,C538847,SET OF 3 BABUSHKA STACKING TINS,-240
1,538848,SET OF 3 BABUSHKA STACKING TINS,240
2,539338,WORLD WAR 2 GLIDERS ASSTD DESIGNS,576
3,539338,60 CAKE CASES DOLLY GIRL DESIGN,240
4,539338,PACK OF 60 SPACEBOY CAKE CASES,240
...,...,...,...
457,578089,RED TOADSTOOL LED NIGHT LIGHT,12
458,578089,POSTAGE,4
459,C578090,POSTAGE,-2
460,580704,VINTAGE DOILY DELUXE SEWING KIT,40


แปลงข้อมูลให้อยู่ในรูปของ Binary Database ที่แต่ละคอลัมน์แทนแต่ละ Item และแต่ละแถวแทนแต่ละ Transaction/Invoice No. ส่วนค่าในแต่ช่องเป็น 1 ถ้าสินค้าชนิดนั้นขายได้ใน Transaction นั้น มิเช่นนั้นเป็น 0

In [3]:
# Transactions done in the United Kingdom
basket = (data.groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

# Defining the hot encoding function to make the data suitable 
# for the concerned libraries
def hot_encode(x):
    return x>= 1
  
# Encoding the datasets
basket = basket.applymap(hot_encode)
basket.head()

Description,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE WOODLAND,3 PIECE SPACEBOY COOKIE CUTTER SET,3 RAFFIA RIBBONS 50'S CHRISTMAS,3 RAFFIA RIBBONS VINTAGE CHRISTMAS,3 TIER CAKE TIN RED AND CREAM,3 TRADITIONAl BISCUIT CUTTERS SET,36 DOILIES DOLLY GIRL,...,WOODEN STAR CHRISTMAS SCANDINAVIAN,WOODEN TREE CHRISTMAS SCANDINAVIAN,WOODLAND CHARLOTTE BAG,WOODLAND SMALL RED FELT HEART,WORLD WAR 2 GLIDERS ASSTD DESIGNS,WRAP VINTAGE DOILY,WRAP ALPHABET DESIGN,WRAP DOLLY GIRL,WRAP RED VINTAGE DOILY,ZINC WILLIE WINKIE CANDLE STICK
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
538848,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
539338,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,True,False,False,False,False,False
540040,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
542428,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
542911,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False


## 3. Learn Association Rules

หากฎความสัมพันธ์ โดยใช้อัลกอริทึม [`apriori`](https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/) โดยเริ่มจากการคำนวณหา Frequent Item

In [4]:
frq_items = apriori(basket, min_support = 0.04, use_colnames = True)

In [5]:
frq_items.sort_values(['support'], ascending=False)

,support,itemsets
50,0.478261,(POSTAGE)
63,0.152174,(SET OF 3 CAKE TINS PANTRY DESIGN )
33,0.130435,(MINI PAINT SET VINTAGE )
53,0.108696,(RED TOADSTOOL LED NIGHT LIGHT)
332,0.108696,"(POSTAGE, RED TOADSTOOL LED NIGHT LIGHT)"
...,...,...
1030,0.043478,"(ASSORTED BOTTLE TOP MAGNETS , 60 CAKE CASES ..."
1031,0.043478,"(ASSORTED BOTTLE TOP MAGNETS , 60 CAKE CASES ..."
1032,0.043478,"(ASSORTED BOTTLE TOP MAGNETS , 60 CAKE CASES ..."
1033,0.043478,"(ASSORTED BOTTLE TOP MAGNETS , 60 CAKE CASES ..."


แล้วใช้ Frequent Item หากฎความสัมพันธ์ 

In [6]:
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)

## 4. Rule Evaluation

ทำการประเมินกฎที่คำนวณได้ด้วยค่า Lift จะพบว่ามีสินค้าบางอันที่มักถูกซื้อพร้อมกันอย่างเช่น  12 PENCILS SMALL TUBE SKULL กับ PACK OF 72 SKULL CAKE CASES ซึ่งอาจจะเป็นไปได้ว่าเวลาทำขนมเป็นของขวัญ ก็ต้องการเครื่องเขียนสำหรับเขียนการ์ดแนบเป็นต้น หรือ CHILDRENS CUTLERY DOLLY GIRL กับ CHILDRENS CUTLERY CIRCUS PARADE ก็อาจจะเป็นไปได้ว่าผู้ซื้อมักจะซื้อเครื่องครัวหลายชุดที่มีลายต่างกัน เป็นต้น

In [7]:
rules.drop(['leverage', 'conviction'], axis=1).sort_values(['confidence', 'lift'], ascending =[False, False])

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift
0,(PACK OF 72 SKULL CAKE CASES),(12 PENCILS SMALL TUBE SKULL),0.043478,0.043478,0.043478,1.000000,23.000000
1,(12 PENCILS SMALL TUBE SKULL),(PACK OF 72 SKULL CAKE CASES),0.043478,0.043478,0.043478,1.000000,23.000000
4,(ASSORTED BOTTLE TOP MAGNETS ),(36 DOILIES DOLLY GIRL),0.043478,0.043478,0.043478,1.000000,23.000000
5,(36 DOILIES DOLLY GIRL),(ASSORTED BOTTLE TOP MAGNETS ),0.043478,0.043478,0.043478,1.000000,23.000000
180,(CHILDRENS CUTLERY DOLLY GIRL ),(CHILDRENS CUTLERY CIRCUS PARADE),0.043478,0.043478,0.043478,1.000000,23.000000
...,...,...,...,...,...,...,...
25487,(POSTAGE),"(WOODEN OWLS LIGHT GARLAND , CHILDRENS CUTLERY...",0.478261,0.043478,0.043478,0.090909,2.090909
202,(POSTAGE),(CUPCAKE LACE PAPER SET 6),0.478261,0.065217,0.043478,0.090909,1.393939
384,(POSTAGE),(MINI PLAYING CARDS DOLLY GIRL ),0.478261,0.065217,0.043478,0.090909,1.393939
483,(POSTAGE),(ROUND SNACK BOXES SET OF4 WOODLAND ),0.478261,0.065217,0.043478,0.090909,1.393939


----